In [1]:
import os
import torch


import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training, AdaLoraConfig, AdaLoraConfig

from transformers import TrainingArguments


In [2]:
import numpy as np
import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

raw_datasets  = load_dataset("glue", 'stsb')


In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification


model_name = "FacebookAI/roberta-base"
config = AutoConfig.from_pretrained(model_name)

config.hidden_dropout_prob=0.0
config.attention_probs_dropout_prob=0.00
#config.num_labels=2
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding

col_to_delete = ['sentence1','sentence2']

def preprocessing_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True,max_length=512)


tokenized_datasets = raw_datasets.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
tokenized_datasets.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [5]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification
from transformers.activations import ACT2FN
import random




In [6]:
from transformers import RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1

)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#model = prepare_model_for_kbit_training(model)

lora_alpha = 5 #16
lora_dropout = 0.05 #0.1
lora_rank = 4 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query",
        "key",
        "value",
        "dense",
    ]
)

model = get_peft_model(model, peft_config)

In [8]:
model = model.to('cuda')


In [9]:
import evaluate
import numpy as np
from sklearn import metrics
import torch
import numpy as np

import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
def compute_metrics(pred):
    preds = np.squeeze(pred.predictions)
    return {"MSE": ((preds - pred.label_ids) ** 2).mean().item(),
            "RMSE": (np.sqrt (( (preds - pred.label_ids) ** 2).mean())).item(),
            "MAE": (np.abs(preds - pred.label_ids)).mean().item(),
            "Pearson" : pearsonr(preds,pred.label_ids)[0],
            "Spearman's Rank":spearmanr(preds,pred.label_ids)[0]
           }

In [10]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='qnli_dir',
    learning_rate=2e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.00,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],

    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:


# Train the model
trainer.train()




Step,Training Loss,Validation Loss,Mse,Rmse,Mae,Pearson,Spearman's rank
100,10.278600,8.005220,8.005220,2.829350,2.399258,0.043047,0.048613
200,5.041000,2.776332,2.776331,1.666233,1.356969,0.705864,0.710354
300,1.220100,0.894615,0.894615,0.945841,0.756487,0.835816,0.844573
400,0.734200,0.599473,0.599473,0.774256,0.593198,0.870994,0.874984
500,0.633200,0.799237,0.799237,0.894001,0.684643,0.868856,0.877953
600,0.540700,0.698345,0.698345,0.835670,0.638912,0.884733,0.890167
700,0.510100,0.693990,0.693990,0.833061,0.636787,0.890224,0.893163
800,0.438400,0.624321,0.624321,0.790140,0.590578,0.885236,0.894225
900,0.419100,0.572517,0.572517,0.756649,0.587421,0.898288,0.898325
1000,0.368700,0.484265,0.484265,0.695891,0.524983,0.904748,0.903571


TrainOutput(global_step=1800, training_loss=1.2607858159806993, metrics={'train_runtime': 285.085, 'train_samples_per_second': 201.659, 'train_steps_per_second': 6.314, 'total_flos': 2123636283570294.0, 'train_loss': 1.2607858159806993, 'epoch': 10.0})